In [1]:
import pandas as pd
import re
from dotenv import load_dotenv


load_dotenv()

True

In [2]:
DATA_DIR = r"../data/uae_real_estate_2024_geo_ready2.parquet"
df =  pd.read_parquet(DATA_DIR)

In [3]:
df.head(5)

,title,displayAddress,bathrooms,bedrooms,addedOn,type,price,verified,priceDuration,sizeMin,furnishing,description,latitude,longitude,City
0,Great ROI I High Floor I Creek View,"Binghatti Canal, Business Bay, Dubai",3,2,2024-08-14 12:02:53+00:00,Residential for Sale,2500000,True,sell,NaN,NO,MNA Properties is delighted to offer this apar...,25.186109,55.286156,Dubai
1,Full Sea View | Beach Life | Brand New Residence,"La Vie, Jumeirah Beach Residence, Dubai",3,2,2024-08-13 05:45:16+00:00,Residential for Sale,7200000,True,sell,NaN,YES,"Apartment for sale in La Vie, Jumeirah Beach R...",25.079735,55.134366,Dubai
2,Fully Furnished | Smart Home | High Floor,"East 40, Al Furjan, Dubai",1,1,2024-08-15 10:05:41+00:00,Residential for Sale,950000,True,sell,NaN,YES,<p>Coldwell banker is pleased to present to yo...,25.021090,55.140395,Dubai
3,VOT / Single Row / Private Pool / Landscaped /...,"Meadows 3, Meadows, Dubai",5,5,2024-08-15 12:58:08+00:00,Residential for Sale,9999990,True,sell,NaN,YES,&#34; 1 Day Prior Notice required &#34; &#34;...,25.070594,55.157207,Dubai
4,Sea View | Move In Ready | Great Location,"Princess Tower, Dubai Marina, Dubai",3,2,2024-08-07 10:32:26+00:00,Residential for Sale,2100000,True,sell,NaN,YES,"GTA Properties is proud to present this 2-bed,...",25.088709,55.146727,Dubai


In [4]:
def fill_nans(df):
    
    df['bathrooms'].fillna(0, inplace=True)
    df['bedrooms'].fillna(0, inplace=True)
    df['furnishing'] = df['furnishing'].cat.add_categories('Unknown')
    df['furnishing'] = df['furnishing'].fillna('Unknown')
    df["sizeMin"] = df["bedrooms"].apply(lambda x: 20 + 12 * x)

In [5]:
def process_time(df):
    
    df['addedOn'] = pd.to_datetime(df['addedOn'])
    df['addedOn_year_month'] = df['addedOn'].dt.strftime('%Y-%m')   # For SQL grouping
    df['addedOn_month_name'] = df['addedOn'].dt.strftime('%B')      # For human-readable output
    df['addedOn_month_num'] = df['addedOn'].dt.month                # For calculations
    df['addedOn_day_num'] = df['addedOn'].dt.day                    # For calculations

In [6]:
process_time(df)
fill_nans(df)

/tmp/ipykernel_104532/2399173419.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['bathrooms'].fillna(0, inplace=True)
/tmp/ipykernel_104532/2399173419.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [7]:
df

,title,displayAddress,bathrooms,bedrooms,addedOn,type,price,verified,priceDuration,sizeMin,furnishing,description,latitude,longitude,City,addedOn_year_month,addedOn_month_name,addedOn_month_num,addedOn_day_num
0,Great ROI I High Floor I Creek View,"Binghatti Canal, Business Bay, Dubai",3,2,2024-08-14 12:02:53+00:00,Residential for Sale,2500000,True,sell,44,NO,MNA Properties is delighted to offer this apar...,25.186109,55.286156,Dubai,2024-08,August,8,14
1,Full Sea View | Beach Life | Brand New Residence,"La Vie, Jumeirah Beach Residence, Dubai",3,2,2024-08-13 05:45:16+00:00,Residential for Sale,7200000,True,sell,44,YES,"Apartment for sale in La Vie, Jumeirah Beach R...",25.079735,55.134366,Dubai,2024-08,August,8,13
2,Fully Furnished | Smart Home | High Floor,"East 40, Al Furjan, Dubai",1,1,2024-08-15 10:05:41+00:00,Residential for Sale,950000,True,sell,32,YES,<p>Coldwell banker is pleased to present to yo...,25.021090,55.140395,Dubai,2024-08,August,8,15
3,VOT / Single Row / Private Pool / Landscaped /...,"Meadows 3, Meadows, Dubai",5,5,2024-08-15 12:58:08+00:00,Residential for Sale,9999990,True,sell,80,YES,&#34; 1 Day Prior Notice required &#34; &#34;...,25.070594,55.157207,Dubai,2024-08,August,8,15
4,Sea View | Move In Ready | Great Location,"Princess Tower, Dubai Marina, Dubai",3,2,2024-08-07 10:32:26+00:00,Residential for Sale,2100000,True,sell,44,YES,"GTA Properties is proud to present this 2-bed,...",25.088709,55.146727,Dubai,2024-08,August,8,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,Upgraded | Corner Unit | Near Lake | Single Row,"Zulal 1, Zulal, The Lakes, Dubai",3,3,2024-06-13 12:54:16+00:00,Residential for Sale,5750000,True,sell,56,NO,The Urban Nest is pleased to present a 3-bedro...,25.079259,55.174432,Dubai,2024-06,June,6,13
559,Marina View | High Floor | Unfurnished,"Cayan Tower, Dubai Marina, Dubai",3,2,2023-09-01 10:43:10+00:00,Residential for Sale,2650000,True,sell,44,NO,Rocky Real Estate is proud to present this mar...,25.086846,55.145171,Dubai,2023-09,September,9,1
560,Invest Now| Amazing Plot| High ROI| Prime Area,"West Yas, Yas Island, Abu Dhabi",0,0,2024-06-21 11:43:02+00:00,Residential for Sale,5200000,False,sell,20,NO,Seize this excellent opportunity to own an ama...,24.494282,54.583130,Abu Dhabi,2024-06,June,6,21
561,Emaar | Waterfront Apartment | 10% DP | 5-Yr PP,"Al Marjan Island, Ras Al Khaimah",2,1,2024-08-12 13:08:33+00:00,Residential for Sale,3007888,False,sell,32,NO,"Discover Address Residence by Emaar, nestled o...",25.677675,55.739598,Ras Al Khaimah,2024-08,August,8,12


In [8]:
# Sample data
# communities = np.array([' Business Bay', ' Jumeirah Beach Residence', ' Al Furjan',
#        ' Meadows', ' Dubai Marina', ' The Villa', ' The Springs',
#        ' Dubai Silicon Oasis', ' Mudon', ' Jumeirah Village Circle',
#        'Hayyan', ' Jebel Ali', ' Reem', ' DAMAC Hills',
#        ' Arabian Ranches', ' Arabian Ranches 2', ' Al Barari',
#        'Khor Fakkan', ' Palm Jumeirah', ' City Walk', ' Downtown Dubai',
#        ' Culture Village', ' Dubai Sports City', ' Muwaileh',
#        ' Dubai Hills Estate', ' Jumeirah', ' The Lakes',
#        ' Jumeirah Village Triangle', 'Downtown Jebel Ali',
#        ' Dubai Residence Complex', 'Al Jubail Island', 'Manama',
#        ' Al Riqqa', ' Al Ghadeer', ' Al Reem Island',
#        ' Discovery Gardens', 'Al Shamkha', 'Muwaileh Commercial',
#        ' Zabeel', ' The Views', ' Muwaileh Commercial', ' DIFC',
#        ' Al Nahda', ' Town Square', ' Meydan', ' Motor City',
#        ' Dubai Festival City', ' Majan', ' Nad Al Sheba', ' Al Jaddaf',
#        ' International City', ' Damac Hills 2', ' Al Marjan Island',
#        ' Al Warsan', 'Wadi Al Safa 5', ' Barsha Heights (Tecom)',
#        ' Al Wasl', ' Al Mamzar - Sharjah', ' Wasl Gate',
#        ' Dubai Industrial City', ' Al Dhait', ' Al Qurm', ' Yas Island',
#        ' Dubai Harbour', 'Al Tallah 2', ' Saadiyat Island', 'Al Helio',
#        'Mina Al Arab', ' Al Quoz', ' The Hills', ' Al Barsha',
#        'Al Mairid', ' Mina Al Arab', ' Al Hamra Village', 'Al Wasl',
#        ' Deira', ' Arjan', 'Al Ameera Village', 'Al Qusaidat',
#        'Seih Al Uraibi', ' Khalifa City', 'Al Zubair', 'Madinat Zayed',
#        ' Al Reef', ' Al Qasimia', 'Al Marjan Island'], dtype=object)

# Define merging rules
merge_rules = {
    # Standardize names
    r'^\s*The\s+(\w+)': r'\1',   # Remove "The" prefix
    r'(\w+)\s+[\-\d]+$': r'\1',  # Remove suffixes like "2", "Commercial"
    r'\(.*\)': '',               # Remove parentheses (e.g., "(Tecom)")
    r'\s*-\s*\w+$': '',          # Remove hyphen suffixes (e.g., "- Sharjah")
    
    # Specific merges
    'Mudon': 'Dubai Land',
    'Jumeirah Village (Circle|Triangle)': 'Jumeirah Village',
    'Downtown Jebel Ali': 'Jebel Ali',
    'Reem|Al Reem Island': 'Al Reem Island',
    'Muwaileh Commercial': 'Muwaileh',
    'DAMAC Hills|Damac Hills 2': 'DAMAC Hills',
    'Arabian Ranches 2': 'Arabian Ranches',
    'Barsha Heights (Tecom)': 'Barsha Heights',
    'Al Mamzar - Sharjah': 'Al Mamzar'
}

def standardize_community(name: str) -> str:
    """Apply merging rules to a community name."""
    if pd.isna(name):
        return name
    
    name = str(name).strip()
    
    # Apply regex replacements
    for pattern, replacement in merge_rules.items():
        name = re.sub(pattern, replacement, name, flags=re.IGNORECASE)
    
    # Final cleanup
    name = name.strip()
    if not name:
        return None
    
    # Capitalize first letters (e.g., "al furjan" → "Al Furjan")
    return ' '.join(word.capitalize() for word in name.split())

# Apply to your array
# standardized = [standardize_community(c) for c in communities]
# unique_communities = pd.Series(standardized).dropna().unique()

# print("Original count:", len(communities))
# print("After standardization:", len(unique_communities))

df["Area"] = df["displayAddress"].apply(lambda x: standardize_community(x.split(",")[-2]))

In [9]:
df["Area"].value_counts()

Area
Dubai Marina         103
Hayyan                40
Jumeirah Village      32
Business Bay          30
Palm Jumeirah         26
                    ... 
Discovery Gardens      1
Al Shamkha             1
Views                  1
Difc                   1
Al Qasimia             1
Name: count, Length: 76, dtype: int64

In [15]:
len(df["Area"].unique())

76

In [16]:
df["Area"] = pd.Categorical(df["Area"])

In [18]:
df["addedOn_month_name"] = pd.Categorical(df["addedOn_month_name"])

In [21]:
df["City"] = pd.Categorical(df["City"])

In [30]:
df["addedOn_year_month"] = df["addedOn_year_month"].astype(str)


In [33]:
all(isinstance(x, str) for x in df["addedOn_year_month"])


True

In [56]:
columns = [
    {"name": "title", "type": "string", "description": "Title of the listing"},
    {"name": "displayAddress", "type": "string", "description": "Full address as displayed"},
    {"name": "bathrooms", "type": "integer", "description": "Number of bathrooms"},
    {"name": "bedrooms", "type": "integer", "description": "Number of bedrooms"},
    {"name": "addedOn", "type": "datetime", "description": "Date and time when added, with timezone"},
    {"name": "type", "type": "string", "description": "Type of the property"},
    {"name": "price", "type": "integer", "description": "Price of the property"},
    {"name": "verified", "type": "boolean", "description": "Whether listing is verified"},
    {"name": "priceDuration", "type": "string", "description": "Price duration type (e.g., per month)"},
    {"name": "sizeMin", "type": "integer", "description": "Minimum size of the property"},
    {"name": "furnishing", "type": "string", "description": "Furnishing status of the property"},
    {"name": "description", "type": "string", "description": "Text description of the listing"},
    {"name": "latitude", "type": "float", "description": "Latitude coordinate"},
    {"name": "longitude", "type": "float", "description": "Longitude coordinate"},
    {"name": "City", "type": "string", "description": "City where the property is located"},
    {"name": "addedOn_year_month", "type": "datetime", "description": "Year and month of added date (YYYY-MM)"},
    {"name": "addedOn_month_name", "type": "string", "description": "Month name of the added date"},
    {"name": "addedOn_month_num", "type": "integer", "description": "Month number of the added date"},
    {"name": "addedOn_day_num", "type": "integer", "description": "Day number of the added date"},
    {"name": "Area", "type": "string", "description": "Area or neighborhood of the property"}
]


In [39]:
df.to_csv(r'../data/clean1.csv', index=False)

In [40]:
import pandasai as pai

In [57]:
df = pai.read_csv('../data/clean1.csv')

In [58]:
datka = pai.create(path="my-org/clean1", df = df, columns=columns)

Dataset saved successfully to path: my-org/clean1
